In [1]:
import numpy as np
import pandas as pd
import os
import re 

posts_full = pd.read_csv('../data/the-reddit-irl-dataset-posts.csv')
comments_full = pd.read_csv('../data/the-reddit-irl-dataset-comments.csv')

posts = posts_full
comments = comments_full

display(posts.head())
display(comments.head())

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ttdab1,2s5ti,meirl,False,1648771149,https://old.reddit.com/r/meirl/comments/ttdab1...,i.imgur.com,https://i.imgur.com/ucZiw34.jpg,NaN,meirl,93
1,post,ttd0r4,2s5ti,meirl,False,1648770296,https://old.reddit.com/r/meirl/comments/ttd0r4...,i.redd.it,https://i.redd.it/x8sehlq23tq81.jpg,NaN,Me_irl,1
2,post,ttd04u,2s5ti,meirl,False,1648770242,https://old.reddit.com/r/meirl/comments/ttd04u...,i.redd.it,https://i.redd.it/p86ehgxw2tq81.jpg,NaN,Meirl,605
3,post,ttc7og,2s5ti,meirl,False,1648767776,https://old.reddit.com/r/meirl/comments/ttc7og...,imgur.com,https://imgur.com/KIU84VZ.jpeg,NaN,meirl,67
4,post,ttc58g,2s5ti,meirl,False,1648767600,https://old.reddit.com/r/meirl/comments/ttc58g...,i.redd.it,https://i.redd.it/to2ot762vsq81.jpg,NaN,me irl,32


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,i2x2j0g,2s5ti,meirl,False,1648771186,https://old.reddit.com/r/meirl/comments/tswh3j...,Yes,NaN,1
1,comment,i2x2hqk,2s5ti,meirl,False,1648771171,https://old.reddit.com/r/meirl/comments/tt9v20...,It’s worse if you’re a man,-0.4767,5
2,comment,i2x2dv0,2s5ti,meirl,False,1648771121,https://old.reddit.com/r/meirl/comments/tt5aas...,They took our jerbs!!!!,0.0000,3
3,comment,i2x2d15,2s5ti,meirl,False,1648771110,https://old.reddit.com/r/meirl/comments/ttbnpo...,It certainly can help you from a mental standp...,0.7650,3
4,comment,i2x28i1,2s5ti,meirl,False,1648771053,https://old.reddit.com/r/meirl/comments/tswh3j...,Bottom picture,0.0000,1


In [2]:
posts = posts_full
comments = comments_full

In [3]:
display(posts.columns)
display(posts.shape)
display(comments.columns)
display(comments.shape)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'domain', 'url', 'selftext', 'title',
       'score'],
      dtype='object')

(3362747, 12)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'body', 'sentiment', 'score'],
      dtype='object')

(12053027, 10)

In [4]:
comments['id_post'] = comments.permalink.str.extract(r'/comments/([^/]+)/', expand=False)
comments['created_hour'] = pd.to_datetime(comments['created_utc'], unit='s', origin='unix').dt.hour

In [5]:
posts_mean_score = posts['score'].mean()
display(posts_mean_score)

240.03112485120053

In [6]:
comments[comments['sentiment'] < -0.9].groupby('created_hour').size().reset_index(name='qty').sort_values(by='qty', ascending=False).head(10)

,created_hour,qty
15,15,2134
16,16,2091
18,18,2044
17,17,2040
20,20,2004
19,19,1996
22,22,1951
14,14,1937
21,21,1869
23,23,1820


In [7]:
posts_to_join = posts[['id', 'selftext', 'url', 'permalink', 'score']]
comments_to_join = comments[['id_post', 'body', 'sentiment', 'created_hour']]
comments_to_join = comments_to_join[(comments['body'].notnull()) & (comments['body'] != 'NaN')]
comments_with_posts = posts_to_join.set_index('id').join(comments_to_join.set_index('id_post'), how='inner')

In [8]:
display(comments_with_posts)

,selftext,url,permalink,score,body,sentiment,created_hour
125chp,NaN,http://i.imgur.com/VFjy4.gif,https://old.reddit.com/r/me_irl/comments/125ch...,15,me too,NaN,22
125upq,NaN,http://i.imgur.com/DDICD.jpg,https://old.reddit.com/r/me_irl/comments/125up...,9,u are a van?,NaN,1
127160,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/12716...,7,rarig,NaN,1
127g42,NaN,http://t2.gstatic.com/images?q=tbn:ANd9GcSHG0e...,https://old.reddit.com/r/me_irl/comments/127g4...,11,:),NaN,14
127g42,NaN,http://t2.gstatic.com/images?q=tbn:ANd9GcSHG0e...,https://old.reddit.com/r/me_irl/comments/127g4...,11,*delicious*,NaN,5
...,...,...,...,...,...,...,...
ttctrk,NaN,https://i.imgur.com/8PZ0BCY.jpg,https://old.reddit.com/r/me_irl/comments/ttctr...,266,[deleted],NaN,23
ttctrk,NaN,https://i.imgur.com/8PZ0BCY.jpg,https://old.reddit.com/r/me_irl/comments/ttctr...,266,[removed],NaN,23
ttcwaq,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/ttcwa...,1,Thanks for posting to me irl! Unfortunately yo...,0.952,23
ttcy2m,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/ttcy2...,1,Thanks for posting to me irl! Unfortunately yo...,0.952,23


In [9]:
student_comments = comments_with_posts[(comments_with_posts['body'].str.contains('university', flags=re.IGNORECASE, regex=True)) |
                                     (comments_with_posts['body'].str.contains('college', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('student', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('teacher', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('professor', flags=re.IGNORECASE, regex=True))]
display(student_comments)

,selftext,url,permalink,score,body,sentiment,created_hour
1mcewm,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/1mcew...,59,oo. oo. ow. student loan debt ow. oo. oh. ow.\...,NaN,1
1weq9d,NaN,http://i.imgur.com/uXjyWyK.jpg,https://old.reddit.com/r/me_irl/comments/1weq9...,76,nahh man Nobody schools Professor Pokemon.,0.0000,12
21lrei,NaN,http://i.imgur.com/7dlVW3X.jpg,https://old.reddit.com/r/me_irl/comments/21lre...,12,I'm that one student who asks whatever questio...,0.0000,21
2cl6nn,NaN,http://37.media.tumblr.com/ef7a2cf29a80da0c1d1...,https://old.reddit.com/r/me_irl/comments/2cl6n...,57,You know why I find this pic iconic? I had to ...,0.0000,20
2dkkbf,NaN,http://i.imgur.com/XifHvBX.gif,https://old.reddit.com/r/me_irl/comments/2dkkb...,49,THIS IS THAT TIME I WENT TO COLLEGE,0.0000,5
...,...,...,...,...,...,...,...
tswh3j,NaN,https://i.redd.it/lxhymxn11pq81.jpg,https://old.reddit.com/r/meirl/comments/tswh3j...,143559,Worst part is I work in a university.. I have ...,-0.7530,17
tt2yio,NaN,https://i.redd.it/2z0sbmwzrqq81.jpg,https://old.reddit.com/r/me_irl/comments/tt2yi...,6630,I must be getting really old to see a movie fr...,0.0000,22
tt6oe5,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/tt6oe...,378,Introverts: What if my voice cracks? What if I...,-0.7033,20
tt7di4,NaN,https://i.imgur.com/MOY62ic.jpg,https://old.reddit.com/r/meirl/comments/tt7di4...,398,And he doesn't have a college degree. Is gross...,-0.8689,20


In [10]:
student_comments_best_urls = student_comments[student_comments['score'] > posts_mean_score]['url']
display(student_comments_best_urls.shape)
display(student_comments_best_urls)

(47025,)

2h2xcc                       http://i.imgur.com/gAGoeVk.png
2hizbm                       http://i.imgur.com/UUuKryW.jpg
2jp72s    http://33.media.tumblr.com/afaa3b156f65b4b817c...
2m914u                             http://imgur.com/6RQotUc
2m914u                             http://imgur.com/6RQotUc
                                ...                        
tswh3j                  https://i.redd.it/lxhymxn11pq81.jpg
tt2yio                  https://i.redd.it/2z0sbmwzrqq81.jpg
tt6oe5                                                  NaN
tt7di4                      https://i.imgur.com/MOY62ic.jpg
tt7g59                  https://i.redd.it/6zz56462srq81.png
Name: url, Length: 47025, dtype: object

In [15]:
student_comments_with_post_score_higher_than_avg = student_comments[student_comments['score'] > posts_mean_score]
student_comments_with_post_score_higher_than_avg.head()

,selftext,url,permalink,score,body,sentiment,created_hour
2h2xcc,NaN,http://i.imgur.com/gAGoeVk.png,https://old.reddit.com/r/me_irl/comments/2h2xc...,3681,Oh ho ho! The student becomes the teacher!,0.0000,4
2hizbm,NaN,http://i.imgur.com/UUuKryW.jpg,https://old.reddit.com/r/me_irl/comments/2hizb...,2427,Am broke college student ;_____;,-0.4215,9
2jp72s,NaN,http://33.media.tumblr.com/afaa3b156f65b4b817c...,https://old.reddit.com/r/me_irl/comments/2jp72...,289,I'm a librarian at a university and this is an...,0.0000,7
2m914u,NaN,http://imgur.com/6RQotUc,https://old.reddit.com/r/me_irl/comments/2m914...,245,"Funny, I'm the opposite. Upon the advent of co...",0.1531,16
2m914u,NaN,http://imgur.com/6RQotUc,https://old.reddit.com/r/me_irl/comments/2m914...,245,Maybe I'm missing the point of this or falsely...,-0.9425,16


In [20]:
student_comments_with_post_score_higher_than_avg.reset_index(inplace=True)

,level_0,post_id,selftext,url,permalink,score,body,sentiment,created_hour
0,0,2h2xcc,NaN,http://i.imgur.com/gAGoeVk.png,https://old.reddit.com/r/me_irl/comments/2h2xc...,3681,Oh ho ho! The student becomes the teacher!,0.0000,4
1,1,2hizbm,NaN,http://i.imgur.com/UUuKryW.jpg,https://old.reddit.com/r/me_irl/comments/2hizb...,2427,Am broke college student ;_____;,-0.4215,9
2,2,2jp72s,NaN,http://33.media.tumblr.com/afaa3b156f65b4b817c...,https://old.reddit.com/r/me_irl/comments/2jp72...,289,I'm a librarian at a university and this is an...,0.0000,7
3,3,2m914u,NaN,http://imgur.com/6RQotUc,https://old.reddit.com/r/me_irl/comments/2m914...,245,"Funny, I'm the opposite. Upon the advent of co...",0.1531,16
4,4,2m914u,NaN,http://imgur.com/6RQotUc,https://old.reddit.com/r/me_irl/comments/2m914...,245,Maybe I'm missing the point of this or falsely...,-0.9425,16
...,...,...,...,...,...,...,...,...,...
47020,47020,tswh3j,NaN,https://i.redd.it/lxhymxn11pq81.jpg,https://old.reddit.com/r/meirl/comments/tswh3j...,143559,Worst part is I work in a university.. I have ...,-0.7530,17
47021,47021,tt2yio,NaN,https://i.redd.it/2z0sbmwzrqq81.jpg,https://old.reddit.com/r/me_irl/comments/tt2yi...,6630,I must be getting really old to see a movie fr...,0.0000,22
47022,47022,tt6oe5,[deleted],NaN,https://old.reddit.com/r/me_irl/comments/tt6oe...,378,Introverts: What if my voice cracks? What if I...,-0.7033,20
47023,47023,tt7di4,NaN,https://i.imgur.com/MOY62ic.jpg,https://old.reddit.com/r/meirl/comments/tt7di4...,398,And he doesn't have a college degree. Is gross...,-0.8689,20


In [26]:
student_comments_with_post_score_higher_than_avg.rename(columns={'index': 'post_id'}, inplace=True)

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [28]:
len(student_comments_with_post_score_higher_than_avg['post_id'].unique())

15033

In [28]:
memes_with_sentiment = comments_with_posts[['url', 'sentiment']].groupby('url').mean().reset_index().dropna().sort_values('sentiment', ascending=False)
display(memes_with_sentiment)

,url,sentiment
237733,https://i.redd.it/2esiak87aje41.jpg,1.0000
1011418,https://i.redd.it/zyvq456cbwz01.png,1.0000
311149,https://i.redd.it/5lpr2qy0hi731.png,1.0000
357659,https://i.redd.it/7mud4j41fvy21.jpg,0.9999
38229,http://i.imgur.com/YdKNs64.jpg,0.9999
...,...,...
490713,https://i.redd.it/de2pshw3r3831.jpg,-0.9965
236149,https://i.redd.it/2c82t2c2z3021.png,-0.9972
103392,http://imgur.com/gxIkznc,-0.9979
1027297,https://i.reddituploads.com/ecc57e12f3424ce4bd...,-0.9979
